# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397853690651                   -0.90    5.2   25.8ms
  2   -8.400240399366       -2.62       -1.74    1.0   18.5ms
  3   -8.400406689561       -3.78       -2.98    1.5   47.7ms
  4   -8.400427850130       -4.67       -2.99    3.0   23.4ms
  5   -8.400427918235       -7.17       -3.02    1.0   18.7ms
  6   -8.400428145226       -6.64       -4.44    1.0   18.9ms
  7   -8.400428151672       -8.19       -4.41    2.2   22.0ms
  8   -8.400428152182       -9.29       -5.24    1.0   18.8ms
  9   -8.400428152207      -10.61       -6.46    1.5   39.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397802890027                   -0.90           5.2    578ms
  2   -8.400385065686       -2.59       -1.79   0.80    2.0    492ms
  3   -8.400423170028       -4.42       -3.01   0.80    1.0    189ms
  4   -8.400428106362       -5.31       -3.43   0.80    2.5   20.2ms
  5   -8.400428149978       -7.36       -4.52   0.80    1.2   16.9ms
  6   -8.400428152176       -8.66       -5.89   0.80    2.0   19.0ms
  7   -8.400428152209      -10.48       -6.12   0.80    3.2   22.2ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.182432787158                   -1.07    4.00s
  2   -1.772645068320        0.47       -0.69    120ms
  3   -4.394453195879        0.42       -0.38   76.8ms
  4   -6.070407069797        0.22       -0.49   44.4ms
  5   -7.518389716472        0.16       -0.68   43.8ms
  6   -7.992591229457       -0.32       -1.22   32.4ms
  7   -8.203621625503       -0.68       -1.51   55.2ms
  8   -8.279330143488       -1.12       -1.70   32.4ms
  9   -8.326103186107       -1.33       -2.16   32.3ms
 10   -8.360281738574       -1.47       -2.37   32

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397832006819                   -0.90    5.2   26.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427981622                   -1.79    28.6s
  2   -8.400428152209       -6.77       -4.03    1.95s
  3   -8.400428152209      -14.45       -7.83    104ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.40967408409478e-7
|ρ_newton - ρ_scfv| = 7.168900063636921e-8
|ρ_newton - ρ_dm|   = 1.427567406616185e-6
